# Use TensorBoard to Debug and Analyze Training Jobs in Amazon SageMaker with PyTorch lightning binary segmentation

Automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many jobs that test a range of hyperparameters on your dataset. You choose the tunable hyperparameters, a range of values for each, and an objective metric. You choose the objective metric from the metrics that the algorithm computes. Automatic model tuning searches the hyperparameters chosen to find the combination of values that result in the model that optimizes the objective metric.

## Introduction

This notebook demonstrates the use of the [PyTorch Segmentation models with pretrained backbones](https://github.com/qubvel/segmentation_models.pytorch) - Python library with Neural Networks for Image Segmentation based on [PyTorch](https://pytorch.org/).

The main features of this library are:

- High level API (just two lines to create a neural network)
- 9 models architectures for binary and multi class segmentation (including legendary Unet)
- 124 available encoders (and 500+ encoders from timm)
- All encoders have pre-trained weights for faster and better convergence
- Popular metrics and losses for training routines

This notebook shows how to use `segmentation-models-pytorch` for **binary** semantic segmentation. We will use the [The Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) (this is an adopted example from Albumentations package [docs](https://albumentations.ai/docs/examples/pytorch_semantic_segmentation/), which is strongly recommended to read, especially if you never used this package for augmentations before).

In [ ]:
!pip install sagemaker -U -q

In [ ]:
from sagemaker.pytorch import PyTorch
from datetime import datetime
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "sagemaker/sagemaker_pytorch_binary_segmentation_intro"


# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

print(f"Using SageMaker version {sagemaker.__version__}")
print(f"Using boto3 version {boto3.__version__}")

In [ ]:
training_dataset_s3_path = "s3://aws-ml-blog/artifacts/amazon-sagemaker-binary-segmentation-intro/oxford-pet-dataset"

In [ ]:
!aws s3 ls {training_dataset_s3_path}/

## Construct a SageMaker training launcher with TensorBoard data configuration

In [ ]:
tensorboard_logs_path = "s3://{}/{}/tensorboard_logs".format(bucket, prefix)
tensorboard_logs_path

In [ ]:
from sagemaker.debugger import TensorBoardOutputConfig

LOG_DIR="/opt/ml/output/tensorboard"

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=tensorboard_logs_path,
    container_local_output_path=LOG_DIR
)

In [ ]:
metric_definitions = [
    {'Name': 'test_dataset_iou', 'Regex': 'test_dataset_iou: ([0-9.]+).*$'},
    {'Name': 'test_per_image_iou', 'Regex': 'test_per_image_iou: ([0-9.]+).*$'},
]

In [ ]:
estimator = PyTorch(entry_point='train.py',
                        source_dir='./code',
                        role=role,
                        framework_version='1.10',
                        py_version='py38',
                        instance_count=1,
                        instance_type='ml.g5.2xlarge',
                        # keep_alive_period_in_seconds=3600,
                        metric_definitions=metric_definitions,
                        tensorboard_output_config=tensorboard_output_config,
                        hyperparameters={
                            'epochs': 3,
                            'arch': "DeepLabV3Plus", ## Unet | FPN | DeepLabV3 | DeepLabV3Plus | Unet \ UnetPlusPlus
                            'tensorboard_logs_path': LOG_DIR
                        })
estimator.fit({"training": training_dataset_s3_path}, logs=True)

## How to access TensorBoard on SageMaker

You can access TensorBoard by two methods: programatically using the sagemaker.interactive_apps.tensorboard module that generates URL, or using the TensorBoard landing page in the SageMaker console. After you open TensorBoard, SageMaker runs the TensorBoard plugin and automatically finds and loads all training job output data in TensorBoard-compatible file format.

In [ ]:
from sagemaker.interactive_apps import tensorboard

region = "us-east-1"
app = tensorboard.TensorBoardApp(region)

print("Navigate to the following URL:")
print(app.get_app_url(training_job_name=estimator.latest_training_job.name))